# Transcript Data Analytics

In [38]:
import os
import json
import typing

In [31]:
ROOT_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
DATA_DIR = os.path.join(ROOT_DIR, 'data', 'podcasts-transcript', 'spotify-podcasts-2020')
TRANSCRIPT_ROOT_DIR = os.path.join(DATA_DIR, 'podcasts-transcripts')

In [32]:
with open(os.path.join(DATA_DIR, 'file_list.txt'), 'r') as f:
    first_line = next(f)

print(first_line)
# first_line = './podcasts-transcripts/4/J/show_4jJdofaAzXkKpsFJ8wGS9I/2i074A63WWvH4Vc279IQiX.json'


./podcasts-transcripts/4/J/show_4Jocfk9mf9D876514gZHet/03R2P2RnGOOZ57hGoXAT6z.json



Pick a sample transcript JSON file

In [20]:
SAMPLE_PATH = os.path.join(DATA_DIR, first_line[2:].replace("\n", ''))

with open(SAMPLE_PATH, 'r') as f:
    data = json.load(f)`

In [21]:
alternatives = data['results'][0]['alternatives']
alternatives[0].keys()

dict_keys([])

## Structure of the transcript JSON file

* The file is a dict contianing a single key `results`, which is a list of items
* Each item in the list is a dictionary that has only one key `alternatives`
* This `alternatives` key refer to a list of **alternative** objects
* Each **alternative** is a dict whose keys include `transcript`, `confidence`, `words`
* `transcript`: an piece of texts corrresponding to a small portion of the transcript of each episode
* `confidence`: double
* `words`: a list of word token objects, each object has attributes `startTime`, `endTime` and `word`

Structure of the transcript JSON

In [22]:
results = data['results']
len(results)

9

### 1. A `result` item

Each `result` item contains a single key called `alternatives` which is a list

In [23]:
result = results[0]
result.keys()

dict_keys(['alternatives'])

### 2. An `alternative` item

Each `alternative` item is dictionary containing three keys
* `transcript`: an excerpt of the transcript
* `confidence`: confidence of the transcript (I guess it indicates how accurate the generated texts are)
* `words`: a list of JSON objects

In [24]:
alternatives = result['alternatives']
len(alternatives)

1

In [25]:
alternative = alternatives[0]
alternative.keys()

dict_keys([])

In [16]:
alternative['transcript']

"Hello and welcome to the first episode of The Fan into flame podcast the official podcast of Greenwood Mennonite School athletics. My name is Tyler warfel. I teach high school math here at GMS and Coach the High School boys basketball team. I'm a big podcast listener. I listen to a lot of different types of podcasts mostly Sports podcast, but also some teaching podcast some news and current events podcasts. I like True Crime podcasts."

In [17]:
alternative['confidence']

0.853115439414978

In [19]:
len(alternative['words'])

76

### 3. A `word` item

Each word item is dict containing 3 required keys `startTime`, `endTime` and `word`. In some alternatives, a word can also contain the `speakerTag`

In [20]:
alternative['words'][:10]

[{'startTime': '1.400s', 'endTime': '1.700s', 'word': 'Hello'},
 {'startTime': '1.700s', 'endTime': '1.800s', 'word': 'and'},
 {'startTime': '1.800s', 'endTime': '2.200s', 'word': 'welcome'},
 {'startTime': '2.200s', 'endTime': '2.400s', 'word': 'to'},
 {'startTime': '2.400s', 'endTime': '2.400s', 'word': 'the'},
 {'startTime': '2.400s', 'endTime': '2.800s', 'word': 'first'},
 {'startTime': '2.800s', 'endTime': '3.500s', 'word': 'episode'},
 {'startTime': '3.500s', 'endTime': '3.600s', 'word': 'of'},
 {'startTime': '3.600s', 'endTime': '3.800s', 'word': 'The'},
 {'startTime': '3.800s', 'endTime': '4.300s', 'word': 'Fan'}]

## Anomaly detection

We suspect that not all **alternatives** have the above structure, let's see if there is something weird

In [21]:
multi_al_results = [res for res in results if len(res['alternatives']) != 1]
multi_al_results

[]

In [24]:
res_alternatives = [res['alternatives'][0] for res in results]
len(res_alternatives)

17

In [26]:
for alt in res_alternatives:
    if 'transcript' not in alt.keys():
        print("Alternative doesn't have `transcript` key")
        print("No. of words: ", len(alt['words']))

Alternative doesn't have `transcript` key
No. of words:  1283


## Conclusion

After the above exploratory analysis, we conclude that:
* Each JSON transcript file hold a single dictionary that contains a single key `results`
* The `results` is a list of dictionary that contains a single key `alternatives` which in tern is a list of object
* Each `alternatives` contains a list of dictionary, dubbed `alternative`
* There are **two** types of `alternative`:
  - Type 1: a dictionary containing **three** keys: `transcript`, `confidence`, and `words`
  - Type 2: a dictionary containing **only one** key `words` that hold all the word tokens in the transcript of a podcast

In [68]:
count = 0
# for (dirpath, dirname, filenames) in os.walk(os.path.join(DATA_DIR, 'podcasts-transcripts')):
#     print(dirname, filenames)
#     count += 1
#     if count > 100:
#         break
json_files = []
PODCAST_DIR = os.path.join(DATA_DIR, 'podcasts-transcripts')
for seg_name in os.listdir(PODCAST_DIR):  # Segments of transcript dataset
    seg_path = os.path.join(PODCAST_DIR, seg_name)
    for alpha_subdir in os.listdir(seg_path):  # alphabet subdirectory
        alpha_subdir_path = os.path.join(seg_path, alpha_subdir)
        for show_dir in os.listdir(alpha_subdir_path):  # show directory
            if show_dir != 'show_0XDDRp9nP5S3kgx413Ixg3':
                continue
            show_path = os.path.join(alpha_subdir_path, show_dir)
            for fname in os.listdir(show_path):
                if not fname.endswith(".json"):
                    continue
                json_files.append(os.path.join(seg_name, alpha_subdir, show_dir, fname))

print("Number of json file", len(json_files))

# with open(os.path.join(DATA_DIR, "json_file_list.txt"), "w") as f:
#     for json_file in json_files:
#         f.write(json_file + "\n")

Number of json file 28


In [70]:
# len(json_files)
json_files[0]

'0/X/show_0XDDRp9nP5S3kgx413Ixg3/3kgNACbCePsmsx1zSMazsZ.json'

## Analyze a sample show id=`0XDDRp9nP5S3kgx413Ixg3`

In [75]:
show_id = '0XDDRp9nP5S3kgx413Ixg3'
show_prefix = 'show_' + show_id
show_dir = os.path.join(DATA_DIR, 'podcasts-transcripts/0/X/show_0XDDRp9nP5S3kgx413Ixg3')
fnames = [fn for fn in os.listdir(show_dir) if fn.endswith(".json")]
len(fnames)

28

In [76]:
def read_json_file(fpath: str) -> typing.Any:
    with open(fpath, 'r') as f:
        data = json.load(f)
    
    return data

def is_empty_alternative(alternatives: typing.List[typing.Any]) -> bool:
    """Check whether an `alternatives` list is empty
    It's empty if it contains a single dictionary
    """
    if len(alternatives) != 1:
        return False
    
    return not bool(alternatives[0])

### Sample episode: id = `1qgr1zTevH7IOvK4My30ht`

Episode 70: Make Your Opponent Hit a First Volley

In [79]:
episode_id = '1qgr1zTevH7IOvK4My30ht'
episode_path = os.path.join(show_dir, episode_id + '.json')

episode = read_json_file(episode_path)

In [78]:
sample_show_json_list = []

with open(os.path.join(DATA_DIR, 'file_list.txt'), 'r') as f:
    for line in f:
        if show_prefix in line.strip():
            sample_show_json_list.append(line.strip())
len(sample_show_json_list)

28

In [82]:
episode['results'][0]['alternatives'][0]

{'transcript': "All I'm egos. Welcome to the a little gray and a lot of Grace podcast. This is your host Ari and the last six years my husband and I have built a million dollar business from our kitchen table and using social media alongside having two babies. So you can only imagine all the value that we want to give you guys. So in this podcast you'll hear from business to how to elevate yourself and become one percent better.",
 'confidence': 0.8736265301704407,
 'words': [{'startTime': '4.700s', 'endTime': '4.900s', 'word': 'All'},
  {'startTime': '4.900s', 'endTime': '5.100s', 'word': "I'm"},
  {'startTime': '5.100s', 'endTime': '5.700s', 'word': 'egos.'},
  {'startTime': '6.600s', 'endTime': '7s', 'word': 'Welcome'},
  {'startTime': '7s', 'endTime': '7.200s', 'word': 'to'},
  {'startTime': '7.200s', 'endTime': '7.600s', 'word': 'the'},
  {'startTime': '7.600s', 'endTime': '7.700s', 'word': 'a'},
  {'startTime': '7.700s', 'endTime': '8s', 'word': 'little'},
  {'startTime': '8s', '

In [83]:
episode_id = '2GW6G1xC9RT4eUDuLgZMB5'
episode_path = os.path.join(show_dir, episode_id + '.json')

episode = read_json_file(episode_path)

In [84]:
episode['results'][0]['alternatives'][0]

{'transcript': "All I'm egos. Welcome to the a little gray and a lot of Grace podcast. This is your host Ari and the last six years my husband and I have built a million dollar business from our kitchen table and using social media alongside having two babies. So you can only imagine all the value that we want to give you guys. So in this podcast you'll hear from business to how to elevate yourself and become one percent better.",
 'confidence': 0.8736265897750854,
 'words': [{'startTime': '4.700s', 'endTime': '4.900s', 'word': 'All'},
  {'startTime': '4.900s', 'endTime': '5.100s', 'word': "I'm"},
  {'startTime': '5.100s', 'endTime': '5.700s', 'word': 'egos.'},
  {'startTime': '6.600s', 'endTime': '7s', 'word': 'Welcome'},
  {'startTime': '7s', 'endTime': '7.200s', 'word': 'to'},
  {'startTime': '7.200s', 'endTime': '7.600s', 'word': 'the'},
  {'startTime': '7.600s', 'endTime': '7.700s', 'word': 'a'},
  {'startTime': '7.700s', 'endTime': '8s', 'word': 'little'},
  {'startTime': '8s', '